# 🎓 GPT-OSS 20B ファインチューニング (Gradio UI版)

このノートブックでは、Gradio UIを使って直感的にモデルの学習ができます。

## 使い方
1. 「**ランタイム**」→「**すべてのセルを実行**」をクリック
2. 最後のセルにUIが表示されます
3. UIの各タブで操作してください

---

<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
</div>

## 📜 ライセンス

このノートブックは [Unsloth](https://github.com/unslothai/unsloth) の
[LGPL-3.0ライセンス](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme) のノートブックを元に作成しました。

**変更点**: Gradio UIの追加、日本語化

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📦 セル1: ライブラリのインストール
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("📦 ライブラリをインストール中...（数分かかります）")

import os, importlib.util
!pip install --upgrade -qqq uv

if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} {get_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@0add68262ab0a2e33b84524346cb27cbb2787356#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo

# Gradioのインストール
!pip install -q gradio

print("✅ インストール完了！")

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 🤖 セル2: モデルの読み込み
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("🤖 モデルを読み込み中...（数分かかります）")

from unsloth import FastLanguageModel
import torch

# グローバル変数
MAX_SEQ_LENGTH = 1024

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",
    dtype = None,
    max_seq_length = MAX_SEQ_LENGTH,
    load_in_4bit = True,
    full_finetuning = False,
)

print("✅ モデル読み込み完了！")

# GPU情報
gpu_stats = torch.cuda.get_device_properties(0)
print(f"🖥️ GPU: {gpu_stats.name}")
print(f"💾 メモリ: {round(gpu_stats.total_memory / 1024 / 1024 / 1024, 1)} GB")

In [ ]:
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 🎨 セル3: Gradio UI
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

import gradio as gr
import json
import shutil
import os
from datasets import Dataset, load_dataset
from trl import SFTConfig, SFTTrainer
from unsloth.chat_templates import standardize_sharegpt, train_on_responses_only

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# グローバル状態
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
state = {
    "dataset": None,
    "trainer": None,
    "model_with_lora": None,
    "training_done": False,
    "logs": []
}

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# データ処理関数
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def load_chatgpt_conversations(file_path):
    """ChatGPTエクスポートを変換"""
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    training_data = []
    for conversation in data:
        messages = []
        mapping = conversation.get('mapping', {})
        for node_id, node in mapping.items():
            msg = node.get('message')
            if msg and msg.get('content', {}).get('parts'):
                role = msg['author']['role']
                parts = msg['content']['parts']
                if parts and isinstance(parts[0], str):
                    content = parts[0].strip()
                    if role in ['user', 'assistant'] and content:
                        messages.append({"role": role, "content": content})
        if len(messages) >= 2:
            training_data.append({"messages": messages})
    return training_data

def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

def load_data(file, use_sample, min_turns, max_turns):
    """データ読み込み"""
    global state
    try:
        if use_sample:
            dataset = load_dataset("HuggingFaceH4/Multilingual-Thinking", split="train")
            dataset = standardize_sharegpt(dataset)
        else:
            if file is None:
                return "❌ ファイルをアップロードしてください"
            raw_data = load_chatgpt_conversations(file.name)
            # フィルタリング
            filtered = [c for c in raw_data if min_turns <= len(c['messages']) <= max_turns]
            dataset = Dataset.from_list(filtered)

        dataset = dataset.map(formatting_prompts_func, batched=True)
        state["dataset"] = dataset
        return f"✅ {len(dataset)} 件の会話を読み込みました"
    except Exception as e:
        return f"❌ エラー: {str(e)}"

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 学習関数
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def start_training(max_steps, learning_rate, batch_size, lora_r, progress=gr.Progress()):
    """学習実行"""
    global state, model

    if state["dataset"] is None:
        return "❌ 先にデータを読み込んでください", None

    try:
        progress(0, desc="LoRAアダプターを設定中...")

        # LoRAアダプター設定
        model_with_lora = FastLanguageModel.get_peft_model(
            model,
            r = int(lora_r),
            target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                              "gate_proj", "up_proj", "down_proj"],
            lora_alpha = 16,
            lora_dropout = 0,
            bias = "none",
            use_gradient_checkpointing = "unsloth",
            random_state = 3407,
        )

        progress(0.1, desc="トレーナーを設定中...")

        # トレーナー設定
        trainer = SFTTrainer(
            model = model_with_lora,
            tokenizer = tokenizer,
            train_dataset = state["dataset"],
            args = SFTConfig(
                per_device_train_batch_size = int(batch_size),
                gradient_accumulation_steps = 4,
                warmup_steps = 5,
                max_steps = int(max_steps),
                learning_rate = float(learning_rate),
                logging_steps = 1,
                optim = "adamw_8bit",
                weight_decay = 0.001,
                lr_scheduler_type = "linear",
                seed = 3407,
                output_dir = "outputs",
                report_to = "none",
            ),
        )

        # 回答部分のみ学習
        trainer = train_on_responses_only(
            trainer,
            instruction_part = "<|start|>user<|message|>",
            response_part = "<|start|>assistant<|channel|>final<|message|>"
        )

        progress(0.2, desc="学習開始...")

        # 学習実行
        trainer_stats = trainer.train()

        state["model_with_lora"] = model_with_lora
        state["training_done"] = True

        # 結果
        runtime = trainer_stats.metrics['train_runtime']
        result = f"""✅ 学習完了！

⏱️ 学習時間: {round(runtime, 1)} 秒 ({round(runtime/60, 1)} 分)
📊 最終Loss: {trainer_stats.metrics.get('train_loss', 'N/A')}
"""
        return result, None

    except Exception as e:
        return f"❌ エラー: {str(e)}", None

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 推論関数
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def test_model(message, reasoning_effort, max_tokens):
    """モデルテスト"""
    global state

    if not state["training_done"]:
        return "⚠️ 先に学習を実行してください"

    try:
        model_to_use = state["model_with_lora"]

        messages = [{"role": "user", "content": message}]
        inputs = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt = True,
            return_tensors = "pt",
            return_dict = True,
            reasoning_effort = reasoning_effort,
        ).to("cuda")

        outputs = model_to_use.generate(
            **inputs,
            max_new_tokens = int(max_tokens),
            use_cache = True,
        )

        response = tokenizer.decode(outputs[0], skip_special_tokens=False)
        # アシスタントの回答部分を抽出
        if "<|start|>assistant" in response:
            response = response.split("<|start|>assistant")[-1]
            response = response.replace("<|channel|>final<|message|>", "").replace("<|end|>", "").replace("<|return|>", "").strip()

        return response
    except Exception as e:
        return f"❌ エラー: {str(e)}"

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 保存関数
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

def save_model(save_location, model_name):
    """モデル保存"""
    global state

    if not state["training_done"]:
        return "⚠️ 先に学習を実行してください", None

    try:
        model_to_save = state["model_with_lora"]

        if save_location == "colab":
            model_to_save.save_pretrained(model_name)
            return f"✅ '{model_name}' に保存しました（Colabセッション終了で消えます）", None

        elif save_location == "gdrive":
            from google.colab import drive
            drive.mount('/content/drive')
            save_path = f"/content/drive/MyDrive/{model_name}"
            model_to_save.save_pretrained(save_path)
            return f"✅ Google Drive に保存しました: {save_path}", None

        elif save_location == "huggingface":
            return "📝 HuggingFaceへの保存は、下記コードを別セルで実行してください:\n\nmodel.push_to_hub('your-username/model-name', token='hf_...')", None

    except Exception as e:
        return f"❌ エラー: {str(e)}", None

def download_model(model_name):
    """モデルをZIPでダウンロード"""
    global state

    if not state["training_done"]:
        return None

    try:
        # 保存
        model_to_save = state["model_with_lora"]
        model_to_save.save_pretrained(model_name)

        # ZIP作成
        zip_path = f"{model_name}.zip"
        shutil.make_archive(model_name, 'zip', model_name)

        return zip_path
    except:
        return None

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Gradio UI
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

with gr.Blocks(title="GPT-OSS 20B Fine-tuning", theme=gr.themes.Soft(), fill_height=True) as demo:
    gr.Markdown("# 🎓 GPT-OSS 20B ファインチューニング")
    gr.Markdown("UIで直感的にモデルを学習できます")

    with gr.Tabs():
        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        # タブ1: データ準備
        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        with gr.Tab("📤 データ準備"):
            gr.Markdown("### データソースを選択")

            use_sample = gr.Checkbox(
                label="サンプルデータを使用（ChatGPTエクスポートがない場合）",
                value=True
            )

            file_input = gr.File(
                label="conversations.json をアップロード",
                file_types=[".json"],
                visible=False
            )

            with gr.Row():
                min_turns = gr.Slider(1, 10, value=2, step=1, label="最小ターン数")
                max_turns = gr.Slider(5, 50, value=20, step=5, label="最大ターン数")

            load_btn = gr.Button("📥 データを読み込む", variant="primary")
            load_result = gr.Textbox(label="結果", interactive=False)

            # サンプルチェックボックスの変更でファイル入力の表示切り替え
            use_sample.change(
                fn=lambda x: gr.update(visible=not x),
                inputs=[use_sample],
                outputs=[file_input]
            )

            load_btn.click(
                fn=load_data,
                inputs=[file_input, use_sample, min_turns, max_turns],
                outputs=[load_result]
            )

        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        # タブ2: 学習設定 & 実行
        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        with gr.Tab("🚀 学習"):
            gr.Markdown("### 学習パラメータ")

            with gr.Row():
                max_steps = gr.Slider(10, 500, value=30, step=10,
                    label="学習ステップ数",
                    info="30: テスト(10分) / 100: 軽め(30分) / 500: しっかり(2時間)")

            with gr.Row():
                learning_rate = gr.Number(value=2e-4, label="学習率", info="推奨: 2e-4")
                batch_size = gr.Slider(1, 4, value=1, step=1, label="バッチサイズ")

            with gr.Row():
                lora_r = gr.Dropdown([8, 16, 32, 64, 128], value=8,
                    label="LoRAランク", info="大きいほど表現力↑、メモリ↑")

            train_btn = gr.Button("🚀 学習を開始", variant="primary", size="lg")
            train_result = gr.Textbox(label="学習結果", interactive=False, lines=5)
            train_plot = gr.Plot(label="学習曲線", visible=False)

            train_btn.click(
                fn=start_training,
                inputs=[max_steps, learning_rate, batch_size, lora_r],
                outputs=[train_result, train_plot]
            )

        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        # タブ3: テスト
        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        with gr.Tab("💬 テスト"):
            gr.Markdown("### 学習したモデルをテスト")

            test_input = gr.Textbox(
                label="メッセージを入力",
                placeholder="こんにちは！自己紹介をしてください。",
                lines=3
            )

            with gr.Row():
                reasoning = gr.Radio(
                    ["low", "medium", "high"],
                    value="medium",
                    label="推論レベル",
                    info="low: 高速 / medium: バランス / high: 詳細"
                )
                max_tokens = gr.Slider(64, 512, value=256, step=64, label="最大トークン数")

            test_btn = gr.Button("💬 送信", variant="primary")
            test_output = gr.Textbox(label="モデルの回答", interactive=False, lines=10)

            test_btn.click(
                fn=test_model,
                inputs=[test_input, reasoning, max_tokens],
                outputs=[test_output]
            )

        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        # タブ4: 保存
        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        with gr.Tab("💾 保存"):
            gr.Markdown("### モデルを保存")

            model_name_input = gr.Textbox(
                label="モデル名",
                value="my_gpt_oss_20b",
                placeholder="my_gpt_oss_20b"
            )

            save_location = gr.Radio(
                ["colab", "gdrive", "huggingface"],
                value="colab",
                label="保存先",
                info="colab: 一時保存 / gdrive: Google Drive / huggingface: HuggingFace Hub"
            )

            save_btn = gr.Button("💾 保存", variant="primary")
            save_result = gr.Textbox(label="結果", interactive=False)

            gr.Markdown("---")
            gr.Markdown("### ダウンロード")

            download_btn = gr.Button("📥 ZIPでダウンロード")
            download_file = gr.File(label="ダウンロード", interactive=False)

            save_btn.click(
                fn=save_model,
                inputs=[save_location, model_name_input],
                outputs=[save_result, download_file]
            )

            download_btn.click(
                fn=download_model,
                inputs=[model_name_input],
                outputs=[download_file]
            )

    gr.Markdown("---")
    gr.Markdown("📜 [LGPL-3.0 License](https://github.com/unslothai/notebooks) | Powered by [Unsloth](https://unsloth.ai)")

# UIを起動（高さを800pxに設定）
demo.launch(share=False, debug=True, height=800)